In [143]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import itertools

from pandas import read_table
%matplotlib inline

In [144]:
#Obtención de datos de las tablas de la base de datos.

items = read_table('lista_jugados_idVideojuego.data', header = None, sep = '|', encoding = 'ISO-8859-1')
items.drop(2, axis = 1, inplace = True)
items.columns = ['game','gamer']

itemsUsuario = read_table('usuario.data', header = None, sep = '|', encoding = 'ISO-8859-1')
itemsUsuario.drop(range(1,5), axis = 1, inplace = True)
itemsUsuario.columns = ['id']

itemsVideojuego = read_table('videojuego.data', header = None, sep = '|', encoding = 'ISO-8859-1')
itemsVideojuego.drop(range(1,6), axis = 1, inplace = True)
itemsVideojuego.columns = ['id']

maxIdUsuario = 0
for i in range(len(itemsUsuario)):
    if maxIdUsuario < itemsUsuario['id'][i]:
        maxIdUsuario = itemsUsuario['id'][i]

tamListaJugados = len(items)
#tamUsuarios = len(itemsUsuario)
tamVideojuegos = len(itemsVideojuego)

C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead.
  import sys
C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead.
  # This is added back by InteractiveShellApp.init_path()


In [145]:
def funcionMaximos(listaMaximos):
    listaReturn = list()
    for i in range(10):
        if len(listaMaximos) == 0:
            break
        listaReturn.append(listaMaximos.index(max(listaMaximos)))
        listaMaximos.pop(listaMaximos.index(max(listaMaximos)))
    return listaReturn

In [146]:
#Resource Allocation.

matrizDatos = np.zeros((tamVideojuegos,maxIdUsuario))
for i in range(tamListaJugados):
    posicionX = items.ix[i].game
    posicionY = items.ix[i].gamer
    matrizDatos[posicionX-1][posicionY-1] = 1

listaDivColumna = np.zeros((tamVideojuegos))
for i in range(tamVideojuegos):
    cantColumna = 0
    for j in range(maxIdUsuario):
        if matrizDatos[i][j] == 1:
            cantColumna += 1
    listaDivColumna[i] = cantColumna
    
listaDivFila = np.zeros((maxIdUsuario))
for i in range(maxIdUsuario):
    cantFila = 0
    for j in range(tamVideojuegos):
        if matrizDatos[j][i] > 0:
            cantFila +=1
    listaDivFila[i] = cantFila

matrizRecomendacion = np.zeros((tamVideojuegos,tamVideojuegos))
desplazaColumna = 0
for i in range(maxIdUsuario):
    if matrizDatos[desplazaColumna][i] == 1:
        for j in range(tamVideojuegos):
            if matrizDatos[j][i] == 1:
                matrizRecomendacion[desplazaColumna][j] = ((1/listaDivColumna[j])/listaDivFila[i])
    desplazaColumna +=1
    
recomendacionesPorJugador = np.zeros((maxIdUsuario,tamVideojuegos))

for i in range(maxIdUsuario):
    listaRecomendacion = list()
    listaIdVideojuego = list()
    for j in range(tamVideojuegos):    
        if matrizDatos[j][i] == 1:
            for k in range(tamVideojuegos):
                if j not in listaIdVideojuego:
                    listaRecomendacion.append(matrizRecomendacion[j][k])
                    listaIdVideojuego.append(j)
    indicesMaximos = funcionMaximos(listaRecomendacion)
    #for a in range(len(listaIdVideojuego)):
        #print(listaIdVideojuego[a])
        
    for x in range(len(indicesMaximos)):
        recomendacionesPorJugador[i][listaIdVideojuego[x]] = 1
        
#for j in range(maxIdUsuario):
    #print(recomendacionesPorJugador[506][j])
            

C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Elvis\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [147]:
#Proyección simple.

matrizUsuariosSimple = np.zeros((maxIdUsuario,maxIdUsuario))

for i in range(maxIdUsuario):
    for j in range(tamVideojuegos):
        if matrizDatos[j][i] == 1:
            for k in range(maxIdUsuario):
                if not k == i:
                    if matrizDatos[j][k] == 1:
                        matrizUsuariosSimple[i, k] += 1

#for i in range(maxIdUsuario):
    #print(matrizUsuariosSimple[i][1])
    
recomendacionesPorJugadorSimple = np.zeros((maxIdUsuario, tamVideojuegos))

for i in range(maxIdUsuario):
    listaUsuariosAAnalizar = list()
    listaRecomendacion = list()
    listaIdVideojuego = list()
    
    for j in range(maxIdUsuario):
        listaUsuariosAAnalizar.append(matrizUsuariosSimple[i][j])
    
    while len(listaUsuariosAAnalizar) != 0:
        if len(listaIdVideojuego) >= 10:
            break
        usuarioMasAfinidad = listaUsuariosAAnalizar.index(max(listaUsuariosAAnalizar))
        listaUsuariosAAnalizar.pop(listaUsuariosAAnalizar.index(max(listaUsuariosAAnalizar)))
        for k in range(tamVideojuegos):
            if matrizDatos[k][usuarioMasAfinidad] == 1:
                if not matrizDatos[k][i] == 1:
                    listaIdVideojuego.append(k+1)
                    
    for x in range(len(listaIdVideojuego)):
        recomendacionesPorJugadorSimple[i][listaIdVideojuego[x]-1] = 1
    
    

In [148]:
#Proyección hiperbólica.

matrizUsuariosHiperbolica = np.zeros((maxIdUsuario,maxIdUsuario))

for i in range(maxIdUsuario):
    for j in range(tamVideojuegos):
        if matrizDatos[j][i] == 1:
            contador = 0
            listaCoincidencias = list()
            for k in range(maxIdUsuario):
                if matrizDatos[j][k] == 1:
                    contador += 1
                    listaCoincidencias.append(k)
            for l in range(len(listaCoincidencias)):
                if not contador == 1:
                    matrizUsuariosHiperbolica[i][listaCoincidencias[l]] += (1/(contador-1))

#for i in range(maxIdUsuario):
    #print(matrizUsuariosSimple[i][1])
    
recomendacionesPorJugadorHiperbolica = np.zeros((maxIdUsuario, tamVideojuegos))

for i in range(maxIdUsuario):
    listaUsuariosAAnalizar = list()
    listaRecomendacion = list()
    listaIdVideojuego = list()
    
    for j in range(maxIdUsuario):
        listaUsuariosAAnalizar.append(matrizUsuariosHiperbolica[i][j])
    
    while len(listaUsuariosAAnalizar) != 0:
        if len(listaIdVideojuego) >= 10:
            break
        usuarioMasAfinidad = listaUsuariosAAnalizar.index(max(listaUsuariosAAnalizar))
        listaUsuariosAAnalizar.pop(listaUsuariosAAnalizar.index(max(listaUsuariosAAnalizar)))
        for k in range(tamVideojuegos):
            if matrizDatos[k][usuarioMasAfinidad] == 1:
                if not matrizDatos[k][i] == 1:
                    listaIdVideojuego.append(k+1)
                    
    for x in range(len(listaIdVideojuego)):
        recomendacionesPorJugadorHiperbolica[i][listaIdVideojuego[x]-1] = 1
    
    

In [149]:
#Filtrar para no recomendar dos veces el mismo juego.
matrizRecomendacionDef = np.zeros((maxIdUsuario,tamVideojuegos))

#Dado que el algoritmo con más probabilidades de acierto es el resourceAllocation se establece 
for i in range(maxIdUsuario):
    for j in range(tamVideojuegos):
        if recomendacionesPorJugadorHiperbolica[i][j] == 1:
            matrizRecomendacionDef[i][j] = 1
        if recomendacionesPorJugadorSimple[i][j] == 1:
            matrizRecomendacionDef[i][j] = 2
        if recomendacionesPorJugador[i][j] == 1:
            matrizRecomendacionDef[i][j] = 3

In [150]:
f=open("insertRA.sql","w")
towrite = []
delete = []
delete.append('DELETE FROM `recomendaciones` WHERE 1;')
f.write(''.join(delete))
f.write('\n')

for i in range(maxIdUsuario):
    for j in range(tamVideojuegos):
        if matrizRecomendacionDef[i][j] > 0:
            towrite = []
            towrite.append('INSERT INTO recomendaciones VALUES (')
            towrite.append(str(i+1))
            towrite.append(',')
            towrite.append(str(j+1))
            towrite.append(',')
            towrite.append(str(matrizRecomendacionDef[i][j]))
            towrite.append(');')   
            f.write(''.join(towrite))
            f.write('\n')
f.close()